In [4]:
from Bio import SeqIO
from IPython.display import FileLink

In [5]:
%load_ext rpy2.ipython

In [18]:
%%R
library(dplyr)

### Interleaving all files

```
!pairs join -t data/6110_5146_27472_AKJ5B_AFPC1_GGACTCCT_CTCTCTAT_R1.fq.gz \
data/6110_5146_27472_AKJ5B_AFPC1_GGACTCCT_CTCTCTAT_R2.fq.gz | pigz -p 20 --fast -c > \
data/Afinterleaved.fq.gz
```

```
!pairs join -t data/6361_5146_27473_AKDNM_AFTF1_TAGGCATG_CTCTCTAT_R1.fq.gz \
data/6361_5146_27473_AKDNM_AFTF1_TAGGCATG_CTCTCTAT_R2.fq.gz | pigz -p 20 --fast -c >> \
data/Afinterleaved.fq.gz
```

In [14]:
!du -h data/Afinterleaved.fq.gz

6.6G	data/Afinterleaved.fq.gz


### Quality filtering, trimming and contamination removal

In [36]:
!bbduk.sh ref=/opt/bbmap/resources/adapters.fa \
in=data/Afinterleaved.fq.gz \
out=data/Afinterleaved.trimmed.fq.gz \
qtrim=rl ktrim=r mink=12 k=23 tbo=t tpe=1 overwrite=true

java -Djava.library.path=/opt/bbmap/jni/ -ea -Xmx13240m -Xms13240m -cp /opt/bbmap/current/ jgi.BBDukF ref=/opt/bbmap/resources/adapters.fa in=data/Afinterleaved.fq.gz out=data/Afinterleaved.trimmed.fq.gz qtrim=rl ktrim=r mink=12 k=23 tbo=t tpe=1 overwrite=true
Executing jgi.BBDukF [ref=/opt/bbmap/resources/adapters.fa, in=data/Afinterleaved.fq.gz, out=data/Afinterleaved.trimmed.fq.gz, qtrim=rl, ktrim=r, mink=12, k=23, tbo=t, tpe=1, overwrite=true]

BBDuk version 37.10
maskMiddle was disabled because useShortKmers=true
Initial:
Memory: max=13304m, free=12888m, used=416m

Added 3262 kmers; time: 	0.068 seconds.
Memory: max=13304m, free=12124m, used=1180m

Input is being processed as unpaired
Started output streams:	0.085 seconds.
Processing time:   		137.591 seconds.

Input:                  	31858278 reads 		7996427778 bases.
QTrimmed:               	17637 reads (0.06%) 	2028394 bases (0.03%)
KTrimmed:               	9682520 reads (30.39%) 	1015359396 bases (12.70%)
Trimmed by overlap: 

In [38]:
!bbduk.sh ref=/opt/bbmap/resources/phix174_ill.ref.fa.gz \
in=data/Afinterleaved.trimmed.fq.gz \
out=data/Afinterleaved.trimmed.contam.fq.gz overwrite=true

java -Djava.library.path=/opt/bbmap/jni/ -ea -Xmx13242m -Xms13242m -cp /opt/bbmap/current/ jgi.BBDukF ref=/opt/bbmap/resources/phix174_ill.ref.fa.gz in=data/Afinterleaved.trimmed.fq.gz out=data/Afinterleaved.trimmed.contam.fq.gz overwrite=true
Executing jgi.BBDukF [ref=/opt/bbmap/resources/phix174_ill.ref.fa.gz, in=data/Afinterleaved.trimmed.fq.gz, out=data/Afinterleaved.trimmed.contam.fq.gz, overwrite=true]

BBDuk version 37.10
Initial:
Memory: max=13306m, free=12890m, used=416m

Added 5360 kmers; time: 	0.057 seconds.
Memory: max=13306m, free=12265m, used=1041m

Input is being processed as unpaired
Started output streams:	0.064 seconds.
Processing time:   		72.500 seconds.

Input:                  	31857235 reads 		6979039988 bases.
Contaminants:           	22428 reads (0.07%) 	5600034 bases (0.08%)
Total Removed:          	22428 reads (0.07%) 	5600034 bases (0.08%)
Result:                 	31834807 reads (99.93%) 	6973439954 bases (99.92%)

Time:   			72.635 seconds.
Reads Processed

### Merging reads

```
!bbmerge.sh in=data/Afinterleaved.trimmed.contam.fq.gz \
out=data/Afinterleaved.trimmed.contam.merged.fq.gz \
outu=data/Afinterleaved.trimmed.contam.unmerged.fq.gz
```

### Normalizaton and error correction

In [40]:
!bbnorm.sh target=50 mindepth=1 ecc=t threads=20 \
in=data/Afinterleaved.trimmed.contam.merged.fq.gz \
out=data/Afinterleaved.trimmed.contam.merged.normed.fq.gz

java -ea -Xmx26434m -Xms26434m -cp /opt/bbmap/current/ jgi.KmerNormalize bits=32 target=50 mindepth=1 ecc=t threads=20 in=data/Afinterleaved.trimmed.contam.merged.fq.gz out=data/Afinterleaved.trimmed.contam.merged.normed.fq.gz
Executing jgi.KmerNormalize [bits=32, target=50, mindepth=1, ecc=t, threads=20, in=data/Afinterleaved.trimmed.contam.merged.fq.gz, out=data/Afinterleaved.trimmed.contam.merged.normed.fq.gz]

BBNorm version 37.10
Set threads to 20

   ***********   Pass 1   **********   


Settings:
threads:          	20
k:                	31
deterministic:    	true
toss error reads: 	false
passes:           	1
bits per cell:    	16
cells:            	9660.14M
hashes:           	3
base min quality: 	5
kmer min prob:    	0.5

target depth:     	200
min depth:        	1
max depth:        	250
min good kmers:   	15
depth percentile: 	64.8
ignore dupe kmers:	true
fix spikes:       	false

Made hash table:  	hashes = 3   	 mem = 17.99 GB   	cells = 9656.47M   	used = 0.872%

Estimated 

In [41]:
!bbnorm.sh target=50 mindepth=1 ecc=t threads=20 \
in=data/Afinterleaved.trimmed.contam.unmerged.fq.gz \
out=data/Afinterleaved.trimmed.contam.unmerged.normed.fq.gz

java -ea -Xmx26526m -Xms26526m -cp /opt/bbmap/current/ jgi.KmerNormalize bits=32 target=50 mindepth=1 ecc=t threads=20 in=data/Afinterleaved.trimmed.contam.unmerged.fq.gz out=data/Afinterleaved.trimmed.contam.unmerged.normed.fq.gz
Executing jgi.KmerNormalize [bits=32, target=50, mindepth=1, ecc=t, threads=20, in=data/Afinterleaved.trimmed.contam.unmerged.fq.gz, out=data/Afinterleaved.trimmed.contam.unmerged.normed.fq.gz]

BBNorm version 37.10
Set threads to 20

   ***********   Pass 1   **********   


Settings:
threads:          	20
k:                	31
deterministic:    	true
toss error reads: 	false
passes:           	1
bits per cell:    	16
cells:            	9693.82M
hashes:           	3
base min quality: 	5
kmer min prob:    	0.5

target depth:     	200
min depth:        	1
max depth:        	250
min good kmers:   	15
depth percentile: 	64.8
ignore dupe kmers:	true
fix spikes:       	false

Made hash table:  	hashes = 3   	 mem = 18.04 GB   	cells = 9685.47M   	used = 11.974%

E

### Megahit assembly & annotation

In [43]:
!megahit -t 20 --12 data/Afinterleaved.trimmed.contam.merged.normed.fq.gz \
--12 data/Afinterleaved.trimmed.contam.unmerged.normed.fq.gz  \
--k-min 21 --k-max 255 --k-step 10 -o data/megahit_outAf

125.812Gb memory in total.
Using: 113.23Gb.
MEGAHIT v1.1.1-2-g02102e1
--- [Thu Aug 17 10:42:04 2017] Start assembly. Number of CPU threads 20 ---
--- [Thu Aug 17 10:42:04 2017] Available memory: 135089143808, used: 121580229427
--- [Thu Aug 17 10:42:04 2017] Converting reads to binaries ---
b'    [read_lib_functions-inl.h  : 209]     Lib 0 (data/Afinterleaved.trimmed.contam.merged.normed.fq.gz): interleaved, 559150 reads, 493 max length'
b'    [read_lib_functions-inl.h  : 209]     Lib 1 (data/Afinterleaved.trimmed.contam.unmerged.normed.fq.gz): interleaved, 2416122 reads, 251 max length'
b'    [utils.h                   : 126]     Real: 11.8352\tuser: 4.1000\tsys: 0.3880\tmaxrss: 162136'
--- [Thu Aug 17 10:42:16 2017] k list: 21,31,41,51,61,71,81,91,101,111,121,131,141,151,161,171,181,191,201,211,221,231,241,251,255 ---
--- [Thu Aug 17 10:42:16 2017] Extracting solid (k+1)-mers for k = 21 ---
--- [Thu Aug 17 10:42:34 2017] Building graph for k = 21 ---
--- [Thu Aug 17 10:42:58 2017] As

In [3]:
!stats.sh data/megahit_outAf/final.contigs.fa

A	C	G	T	N	IUPAC	Other	GC	GC_stdev
0.2978	0.2072	0.2056	0.2893	0.0000	0.0000	0.0000	0.4128	0.0780

Main genome scaffold total:         	51354
Main genome contig total:           	51354
Main genome scaffold sequence total:	31.308 MB
Main genome contig sequence total:  	31.308 MB  	0.000% gap
Main genome scaffold N/L50:         	12233/541
Main genome contig N/L50:           	12233/541
Main genome scaffold N/L90:         	41913/353
Main genome contig N/L90:           	41913/353
Max scaffold length:                	17.548 KB
Max contig length:                  	17.548 KB
Number of scaffolds > 50 KB:        	0
% main genome in scaffolds > 50 KB: 	0.00%


Minimum 	Number        	Number        	Total         	Total         	Scaffold
Scaffold	of            	of            	Scaffold      	Contig        	Contig  
Length  	Scaffolds     	Contigs       	Length        	Length        	Coverage
--------	--------------	--------------	--------------	--------------	--------
    All 	        51,354	       

In [47]:
!bioawk -c fastx '{if ( length($seq) >= 500 ) print ">"$name"\n"$seq }' \
data/megahit_outAf/final.contigs.fa > data/megahit_outAf/final.contigs.gt500.fa

In [49]:
!tblastx -db data/ssDNAviruses \
-query data/megahit_outAf/final.contigs.gt500.fa \
-outfmt 6 -evalue 0.00000001 -max_target_seqs 5 -num_threads 25 \
-out data/megahit_outAf/DNAvirusesPO.m6

In [28]:
!tblastx -db /home/elliot/assembly/RNAmetaviromes/data/RNAvirusdatabase  \
-query data/megahit_outAf/final.contigs.gt500.fa \
-outfmt 6 -evalue 0.00000001 -max_target_seqs 5 -num_threads 10 \
-out data/megahit_outAf/RNAviruses.m6

### Isolating contigs and mapping reads to them

In [85]:
%%R

options(dplyr.width=Inf)

df = read.csv("data/megahit_outAf/DNAvirusesPO.m6", sep = "\t",
         stringsAsFactors = FALSE, header = FALSE) %>%
    group_by(V1) %>%
    filter(V2 =="Sea_star_associated_densovirus-like_genome_fragment") %>%
    filter(rank(-V12, ties.method = "random") == 1)  %>%
    filter(V4 >=10) %>%
    select(V1, V2, V3, V4, V12, V11)

ids = df$V1
df

Source: local data frame [5 x 6]
Groups: V1 [5]

          V1                                                  V2    V3    V4
       <chr>                                               <chr> <dbl> <int>
1 k255_18196 Sea_star_associated_densovirus-like_genome_fragment 90.24   379
2 k255_20446 Sea_star_associated_densovirus-like_genome_fragment 54.17    48
3 k255_28005 Sea_star_associated_densovirus-like_genome_fragment 79.69    64
4 k255_34599 Sea_star_associated_densovirus-like_genome_fragment 73.96    96
5 k255_43172 Sea_star_associated_densovirus-like_genome_fragment 84.21   114
    V12    V11
  <dbl>  <dbl>
1 870.0  0e+00
2  58.3  1e-12
3 124.0  6e-30
4 177.0  9e-46
5 204.0 5e-104


In [54]:
ids = %Rget ids

In [57]:
keepers= []

for rec in SeqIO.parse("data/megahit_outAf/final.contigs.gt500.fa", "fasta"):
    if rec.name in ids:
        print(rec.name, len(rec.seq))
        keepers.append(rec)

with open("data/megahit_outAf/seastar_hit_contigs.fa", "w") as out_handle:
    SeqIO.write(keepers, out_handle, "fasta")

k255_18196 5220
k255_20446 629
k255_28005 681
k255_34599 556
k255_43172 1185


In [62]:
!bbmap.sh mappedonly=t minid=0.95 idfilter=0.98 \
in=data/Afinterleaved.trimmed.contam.fq.gz \
ref=data/megahit_outAf/seastar_hit_contigs.fa \
out=data/megahit_outAf/reads2megahit.fq \
threads=15

java -Djava.library.path=/opt/bbmap/jni/ -ea -Xmx26490m -cp /opt/bbmap/current/ align2.BBMap build=1 overwrite=true fastareadlen=500 mappedonly=t minid=0.95 idfilter=0.98 in=data/Afinterleaved.trimmed.contam.fq.gz ref=data/megahit_outAf/seastar_hit_contigs.fa out=data/megahit_outAf/reads2megahit.fq threads=15
Executing align2.BBMap [build=1, overwrite=true, fastareadlen=500, mappedonly=t, minid=0.95, idfilter=0.98, in=data/Afinterleaved.trimmed.contam.fq.gz, ref=data/megahit_outAf/seastar_hit_contigs.fa, out=data/megahit_outAf/reads2megahit.fq, threads=15]

BBMap version 37.10
Set OUTPUT_MAPPED_ONLY to true
Set threads to 15
Retaining first best site only for ambiguous mappings.
Set MINIMUM_ALIGNMENT_SCORE_RATIO to 0.908
NOTE:	Ignoring reference file because it already appears to have been processed.
NOTE:	If you wish to regenerate the index, please manually delete ref/genome/1/summary.txt
Set genome to 1

Loaded Reference:	0.081 seconds.
Loading index for chunk 1-1, build 1
Generated 

In [75]:
!bbnorm.sh threads=15 \
in=data/megahit_outAf/reads2megahit.fq  \
target=80 min=1 aec=t \
out=data/megahit_outAf/reads2megahit.normed.fq 

java -ea -Xmx26501m -Xms26501m -cp /opt/bbmap/current/ jgi.KmerNormalize bits=32 threads=15 in=data/megahit_outAf/reads2megahit.fq target=80 min=1 aec=t out=data/megahit_outAf/reads2megahit.normed.fq
Executing jgi.KmerNormalize [bits=32, threads=15, in=data/megahit_outAf/reads2megahit.fq, target=80, min=1, aec=t, out=data/megahit_outAf/reads2megahit.normed.fq]

BBNorm version 37.10
Set threads to 15

   ***********   Pass 1   **********   


Settings:
threads:          	15
k:                	31
deterministic:    	true
toss error reads: 	false
passes:           	1
bits per cell:    	16
cells:            	9685.02M
hashes:           	3
base min quality: 	5
kmer min prob:    	0.5

target depth:     	320
min depth:        	1
max depth:        	400
min good kmers:   	15
depth percentile: 	64.8
ignore dupe kmers:	true
fix spikes:       	false

Made hash table:  	hashes = 3   	 mem = 18.03 GB   	cells = 9681.01M   	used = 0.003%

Estimated unique kmers:     	92682

Table creation time:		37.818

### Spades assembly to mapped reads

In [76]:
!spades.py --thread 20 --12 data/megahit_outAf/reads2megahit.normed.fq   \
-o data/spades_Af_denso >/dev/null

In [ ]:
!bbmap.sh threads=15 mappedonly=t minid=0.95 idfilter=0.98 \
in=data/Afinterleaved.trimmed.contam.fq.gz \
ref=data/spades_Af_denso/contigs.fasta \
covstats=data/spades_Af_denso/covstats \
basecov=data/spades_Af_denso/bascov.txt \
covhist=data/spades_Af_denso/hicov.txt

In [40]:
##Original read mapping
!bbmap.sh threads=15 mappedonly=t minid=0.95 idfilter=0.98 \
in=data/Afinterleaved.trimmed.contam.fq.gz \
ref=data/spades_Af_denso/contigs.fasta \
out=data/spades_Af_denso/mappedreadsAF.fq

java -Djava.library.path=/opt/bbmap/jni/ -ea -Xmx32599m -cp /opt/bbmap/current/ align2.BBMap build=1 overwrite=true fastareadlen=500 threads=15 mappedonly=t minid=0.95 idfilter=0.98 in=data/Afinterleaved.trimmed.contam.fq.gz ref=data/spades_Af/contigs.fasta out=data/spades_Af/mappedreadsAF.fq
Executing align2.BBMap [build=1, overwrite=true, fastareadlen=500, threads=15, mappedonly=t, minid=0.95, idfilter=0.98, in=data/Afinterleaved.trimmed.contam.fq.gz, ref=data/spades_Af/contigs.fasta, out=data/spades_Af/mappedreadsAF.fq]

BBMap version 37.10
Set threads to 15
Set OUTPUT_MAPPED_ONLY to true
Retaining first best site only for ambiguous mappings.
Set MINIMUM_ALIGNMENT_SCORE_RATIO to 0.908
NOTE:	Ignoring reference file because it already appears to have been processed.
NOTE:	If you wish to regenerate the index, please manually delete ref/genome/1/summary.txt
Set genome to 1

Loaded Reference:	0.064 seconds.
Loading index for chunk 1-1, build 1
Generated Index:	0.589 seconds.
Analyzed Ind

In [1]:
##Isolating High Quality Reads
!bbduk.sh ref=/opt/bbmap/resources/adapters.fa \
in=data/Afinterleaved.trimmed.contam.fq.gz \
out=data/Afinterleaved.trimmed.contam.HQ.fq.gz \
k=30 tbo tpe qtrim=rl trimq=30 overwrite=true

java -Djava.library.path=/opt/bbmap/jni/ -ea -Xmx12751m -Xms12751m -cp /opt/bbmap/current/ jgi.BBDukF ref=/opt/bbmap/resources/adapters.fa in=data/Afinterleaved.trimmed.contam.fq.gz out=data/Afinterleaved.trimmed.contam.HQ.fq.gz k=30 tbo tpe qtrim=rl trimq=30 overwrite=true
Executing jgi.BBDukF [ref=/opt/bbmap/resources/adapters.fa, in=data/Afinterleaved.trimmed.contam.fq.gz, out=data/Afinterleaved.trimmed.contam.HQ.fq.gz, k=30, tbo, tpe, qtrim=rl, trimq=30, overwrite=true]

BBDuk version 37.10
Initial:
Memory: max=12814m, free=12346m, used=468m

Added 2957 kmers; time: 	0.218 seconds.
Memory: max=12814m, free=11677m, used=1137m

Input is being processed as unpaired
Started output streams:	0.268 seconds.
Processing time:   		96.526 seconds.

Input:                  	31834807 reads 		6973439954 bases.
Contaminants:           	20530 reads (0.06%) 	5035226 bases (0.07%)
QTrimmed:               	23162806 reads (72.76%) 	2827373447 bases (40.54%)
Trimmed by overlap:     	0 reads (0.00%) 	0 

In [2]:
##Fastq file output
!bbmap.sh threads=15 mappedonly=t minid=0.95 idfilter=0.98 \
in=data/Afinterleaved.trimmed.contam.HQ.fq.gz \
ref=data/spades_Af_denso/contigs.fasta \
out=data/spades_Af_denso/mappedreadsAF.HQ.fq

java -Djava.library.path=/opt/bbmap/jni/ -ea -Xmx26086m -cp /opt/bbmap/current/ align2.BBMap build=1 overwrite=true fastareadlen=500 threads=15 mappedonly=t minid=0.95 idfilter=0.98 in=data/Afinterleaved.trimmed.contam.HQ.fq.gz ref=data/spades_Af/contigs.fasta out=data/spades_Af/mappedreadsAF.HQ.fq
Executing align2.BBMap [build=1, overwrite=true, fastareadlen=500, threads=15, mappedonly=t, minid=0.95, idfilter=0.98, in=data/Afinterleaved.trimmed.contam.HQ.fq.gz, ref=data/spades_Af/contigs.fasta, out=data/spades_Af/mappedreadsAF.HQ.fq]

BBMap version 37.10
Set threads to 15
Set OUTPUT_MAPPED_ONLY to true
Retaining first best site only for ambiguous mappings.
Set MINIMUM_ALIGNMENT_SCORE_RATIO to 0.908
NOTE:	Ignoring reference file because it already appears to have been processed.
NOTE:	If you wish to regenerate the index, please manually delete ref/genome/1/summary.txt
Set genome to 1

Loaded Reference:	0.161 seconds.
Loading index for chunk 1-1, build 1
Generated Index:	0.592 seconds.
